In [ ]:
import argparse
import json

import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions
from apache_beam.transforms.window import FixedWindows

from apache_beam.io.kafka import ReadFromKafka

# Define opciones de pipeline
options = PipelineOptions()

# Define opciones de configuración
setup_options = options.view_as(SetupOptions)
setup_options.setup_file = './setup.py'

# Define opciones de Kafka
kafka_options = {
    "bootstrap_servers": "127.0.0.1:9092",  # Dirección IP y puerto de Kafka
    "topics": ["test_topic"],  # Nombre del topic de Kafka
    "consumer_config": {"group.id": "my-group"}  # Configuración de consumidor de Kafka
}

# Define una función para procesar mensajes de Kafka
def process_message(message):
    # Decodifica el mensaje
    message_str = message.value.decode('utf-8')

    # Carga el mensaje como un objeto JSON
    message_json = json.loads(message_str)

    # Procesa el mensaje
    # En este ejemplo, simplemente imprimimos el mensaje en la consola
    print(message_json)

# Define el pipeline
with beam.Pipeline(options=options) as p:
    # Lee datos de Kafka utilizando el transform ReadFromKafka
    # Este transform lee datos de Kafka en formato PCollection
    kafka_data = (p
                  | "ReadFromKafka" >> ReadFromKafka(**kafka_options)
                  | "ProcessData" >> beam.Map(process_message))
